# One gene id per hugo symbol

In [67]:
import pandas as pd

In [68]:
et = pd.read_csv("../data/ensembl_biomart_transcripts.txt", sep='\t')

In [69]:
et_hugo_counts = et.groupby("hgnc_symbol").gene_stable_id.nunique()
et_hugo_counts.head()

hgnc_symbol
A1BG        1
A1BG-AS1    1
A1CF        1
A2M         1
A2M-AS1     1
Name: gene_stable_id, dtype: int64

In [4]:
len(et_hugo_counts[et_hugo_counts > 1])

2004

In [5]:
et_geneid_counts = et.groupby("gene_stable_id").hgnc_symbol.nunique()
et_geneid_counts.head()

gene_stable_id
ENSG00000000003    1
ENSG00000000005    1
ENSG00000000419    1
ENSG00000000457    1
ENSG00000000460    1
Name: hgnc_symbol, dtype: int64

In [6]:
len(et_geneid_counts[et_geneid_counts > 1])

70

In [7]:
et_geneid_counts[et_geneid_counts > 1].head()

gene_stable_id
ENSG00000109072    2
ENSG00000199480    2
ENSG00000199654    2
ENSG00000207688    2
ENSG00000207704    2
Name: hgnc_symbol, dtype: int64

In [8]:
multiple_ensembl = set(list(et_hugo_counts[et_hugo_counts > 1].index))

In [9]:
et_hugo_counts[et_hugo_counts > 1].head()

hgnc_symbol
AADACL2     2
ABCA11P     2
ABCB10P4    2
ABCB11      2
ABCD1       2
Name: gene_stable_id, dtype: int64

In [10]:
len(et_hugo_counts)

33670

In [11]:
!ls ../data

20180321_hgnc_symbols.txt
Homo_sapiens.GRCh37.gff3.gz
Makefile
ensembl_biomart_canonical_transcripts_per_hgnc.txt
ensembl_biomart_geneids_grch37.p13.transcript_info.txt
ensembl_biomart_geneids_grch37.p13.txt
ensembl_biomart_pfam_grch37.p13.txt
ensembl_biomart_transcripts.json
ensembl_biomart_transcripts.json.gz
ensembl_biomart_transcripts.txt
ensembl_exon_info.txt
hgnc.txt
isoform_overrides_at_mskcc.txt
isoform_overrides_genome_nexus.txt
isoform_overrides_uniprot.txt
oncokb_cancer_genes_list_20170926.txt
pfamA.txt


In [12]:
hugo = pd.read_csv("../data/20180321_hgnc_symbols.txt", sep="\t")

In [13]:
# 1912 of 2004 are approved symbols, so 86 are not approved (maybe synonyms?)
len(hugo[hugo['Approved Symbol'].isin(multiple_ensembl)])

1912

In [14]:
# no ensembl id assigned in hgnc website
hugo_override = hugo[hugo['Approved Symbol'].isin(multiple_ensembl)].set_index('Approved Symbol')['Ensembl ID(supplied by Ensembl)']
print("Approved symbols w/o ensembl id in hugo, but multiple in ensembl dump: ", pd.isnull(hugo_override).sum())
# TODO: check whether all these are in ensembl transcripts file
hugo_override[pd.isnull(hugo_override)]

Approved symbols w/o ensembl id in hugo, but multiple in ensembl dump:  17


Approved Symbol
AHCYP1        NaN
CCL4L1        NaN
CTSLP3        NaN
DUSP8P1       NaN
DUSP8P2       NaN
FAM83H-AS1    NaN
FAM226A       NaN
GLUD1P6       NaN
HIST2H2BC     NaN
LILRA3        NaN
RNA5-8S5      NaN
RNVU1-8       NaN
RNVU1-11      NaN
RPL17P42      NaN
TRBV6-9       NaN
TRBV7-8       NaN
VN1R14P       NaN
Name: Ensembl ID(supplied by Ensembl), dtype: object

In [15]:
print("{} ensembl ids assigned to hugo symbols from {} total (+ {} w/o assignment) that are not in ensembl transcript dump".format(
    ((~hugo['Ensembl ID(supplied by Ensembl)'].isin(et.gene_stable_id.unique())) & (~(pd.isnull(hugo['Ensembl ID(supplied by Ensembl)'])))).sum(),
    (~pd.isnull(hugo['Ensembl ID(supplied by Ensembl)'])).sum(),
    pd.isnull(hugo['Ensembl ID(supplied by Ensembl)']).sum())
)

1572 ensembl ids assigned to hugo symbols from 37253 total (+ 8491 w/o assignment) that are not in ensembl transcript dump


In [69]:
# TODO: check of those without assignemnt if they are in ensembl dump
print("Of the {} ones w/o assignment, there are {} associated transcripts in ensembl dump".format(
    pd.isnull(hugo['Ensembl ID(supplied by Ensembl)']).sum(),
    hugo['Approved Symbol'][pd.isnull(hugo['Ensembl ID(supplied by Ensembl)'])].isin(et.hgnc_symbol).sum()
)
)

Of the 8491 ones w/o assignment, there are 222 associated transcripts in ensembl dump


In [18]:
# not that many important ones, but good to resolve
hugo[pd.isnull(hugo['Ensembl ID(supplied by Ensembl)']) & (hugo['Approved Symbol'].isin(et.hgnc_symbol))]

,HGNC ID,Approved Symbol,Approved Name,Status,Previous Symbols,Synonyms,Chromosome,Accession Numbers,RefSeq IDs,Entrez Gene ID(supplied by NCBI),RefSeq(supplied by NCBI),UniProt ID(supplied by UniProt),Ensembl ID(supplied by Ensembl)
725,HGNC:24620,AHCTF1P1,AT-hook containing transcription factor 1 pseu...,Approved,AHCTF1P,ELYS-like,2q24.2,NaN,NG_002622,285116.0,NR_077058,NaN,NaN
729,HGNC:44993,AHCYP1,adenosylhomocysteinase pseudogene 1,Approved,NaN,NaN,10q11.22,NaN,NG_022175,340844.0,NG_022175,NaN,NaN
1143,HGNC:43603,ANKRD20A12P,"ankyrin repeat domain 20 family member A12, ps...",Approved,NaN,NaN,4,NaN,NaN,100874392.0,NR_046228,Q8NF67,NaN
1145,HGNC:43605,ANKRD20A14P,"ankyrin repeat domain 20 family member A14, ps...",Approved,NaN,NaN,unplaced,NaN,NaN,100533719.0,NG_028834,NaN,NaN
1668,HGNC:44201,ARL14EPL,ADP ribosylation factor like GTPase 14 effecto...,Approved,NaN,NaN,5q23.1,NaN,NM_001195581,644100.0,NM_001195581,P0DKL9,NaN
1842,HGNC:39401,ASNSP5,asparagine synthetase pseudogene 5,Approved,NaN,NaN,unplaced,NaN,NaN,100873794.0,NG_032287,NaN,NaN
2113,HGNC:39662,ATP8A2P1,ATPase phospholipid transporting 8A2 pseudogene 1,Approved,NaN,NaN,10p11.21,NaN,NG_025483,100422505.0,NG_025483,NaN,NaN
2300,HGNC:15732,BAGE5,BAGE family member 5,Approved,NaN,CT2.5,"13cen, GRCh38 novel patch",AF339516,NM_182484,85316.0,NM_182484,Q86Y27,NaN
2622,HGNC:19436,BMS1P18,"BMS1, ribosome biogenesis factor pseudogene 18",Approved,"C14orf17, LINC00516",NaN,14q11.2,BC040855,NaN,414763.0,NR_073459,NaN,NaN
2624,HGNC:49153,BMS1P20,"BMS1, ribosome biogenesis factor pseudogene 20",Approved,NaN,NaN,22q11.22,NaN,NaN,96610.0,NR_027293,NaN,NaN


In [ ]:
# TODO: check synonyms

In [39]:
# check mskcc isoforms if they provide solutions for the 2K
mskcc = pd.read_csv('../data/isoform_overrides_at_mskcc.txt', sep='\t')
len(mskcc[mskcc.gene_name.isin(multiple_ensembl)])

21

In [40]:
uniprot = pd.read_csv('../data/isoform_overrides_uniprot.txt', sep='\t')
len(uniprot[uniprot.gene_name.isin(multiple_ensembl)])

1024

In [81]:
hugo[pd.isnull(hugo['Ensembl ID(supplied by Ensembl)'])].Status.value_counts()

Approved            4056
Symbol Withdrawn    3268
Entry Withdrawn     1167
Name: Status, dtype: int64

In [84]:
hugo_approved_missing_ensembl_id = hugo[pd.isnull(hugo['Ensembl ID(supplied by Ensembl)']) & (hugo.Status == "Approved")]

In [97]:
print("There are genes in ensembl transcript for {}".format((hugo_approved_missing_ensembl_id['Approved Symbol'].isin(set(et.hgnc_symbol.unique()))).sum()))

There are genes in ensembl transcript for 222


In [93]:
hugo_approved_missing_eid = hugo_approved_missing_ensembl_id[~(hugo_approved_missing_ensembl_id['Approved Symbol'].isin(set(et.hgnc_symbol.unique())))]

In [96]:
len(hugo_approved_missing_eid)

3834

See also [gdocs](https://docs.google.com/spreadsheets/d/12o3OhlT-K0XQHW6T_kRxCaWIwwUyYlUY5OZkQP9wwns/edit#gid=0)

so i looked at some of those genes, but basically a lot of them don't have genomic coordinates in genecards for whatever reason. Some are e.g. not on the reference assembly or on some alternate reference locus.

I was able to find genomic coordinates for this one http://www.genecards.org/Search/Keyword?queryString=ZNF726P1 and look at vep output. But all the geneids outputted by vep are different:
http://rest.ensembl.org/vep/human/hgvs/19:g.23832282-23832470A%3EC?content-type=application/json

also using xrefs endpoint from ensembl gives nothing:
http://rest.ensembl.org/xrefs/symbol/homo_sapiens/ZNF726P1?content-type=application/json

so yeah it seems these genes are all not super important

# Validating canonical output

In [70]:
can = pd.read_csv("../export/ensembl_biomart_canonical_transcripts_per_hgnc.txt", sep="\t")

In [71]:
# check if all canonical transcripts are in the ensembl data dump
all_transcripts_in_dump = set(et.transcript_stable_id.unique())
can['mskcc_not_in_dump'] = can.mskcc_canonical_transcript.apply(lambda x: not pd.isnull(x) and x not in all_transcripts_in_dump)
can['uniprot_not_in_dump'] = can.uniprot_canonical_transcript.apply(lambda x: not pd.isnull(x) and x not in all_transcripts_in_dump)
print("Missing mskcc transcripts from dump: {}".format((can.mskcc_not_in_dump & (can.uniprot_canonical_transcript != can.mskcc_canonical_transcript)).sum()))
print("Missing uniprot transcripts from dump: {}".format((can.uniprot_not_in_dump).sum()))
can['ensembl_not_in_dump'] = can.ensembl_canonical_transcript.apply(lambda x: not pd.isnull(x) and x not in all_transcripts_in_dump)
print("Missing ensembl transcripts from dump (should be none): {}".format(can.ensembl_not_in_dump.sum()))
can[can.uniprot_not_in_dump].to_clipboard(index=False)

Missing mskcc transcripts from dump: 0
Missing uniprot transcripts from dump: 308
Missing ensembl transcripts from dump (should be none): 0


Added these to [gdocs](https://docs.google.com/spreadsheets/d/12o3OhlT-K0XQHW6T_kRxCaWIwwUyYlUY5OZkQP9wwns/edit#gid=0). We could handle in genome nexus backend by returning ensembl_canonical_transcript when missing or on frontend.

In [9]:
# check if all canonical genes are in the ensembl data dump
all_ensembl_genes_in_dump = set(et.gene_stable_id.unique())
can['canonical_gene_in_dump'] = can.ensembl_canonical_gene.apply(lambda x: not pd.isnull(x) and x not in all_ensembl_genes_in_dump)
print("Missing ensembl genes from dump (should be none): {}".format(can.canonical_gene_in_dump.sum()))

Missing ensembl genes from dump (should be none): 0


# Propose new uniprot ids for hg19

In [72]:
uniprot = pd.read_csv("../data/isoform_overrides_uniprot.txt",sep="\t")

In [73]:
uniprot.columns

Index(['enst_id', 'gene_name', 'refseq_id', 'ccds_id'], dtype='object')

In [74]:
unimerge = pd.merge(uniprot, 
                         can["hgnc_symbol uniprot_not_in_dump uniprot_canonical_transcript ensembl_canonical_transcript".split()],
                         how='left',
                         left_on=['gene_name'],
                         right_on=['hgnc_symbol'],
                         )

In [75]:
print("Some uniprot entries hugo symbol don't match, because using old hugo symbols: {}".format(
    unimerge.hgnc_symbol.isnull().sum())
)
print("Could fix this by mapping to approved symbols, but not important for this purpose since all those are in dump")
assert(len(unimerge[unimerge.hgnc_symbol.isnull() & unimerge.uniprot_not_in_dump]) == 0)

Some uniprot entries hugo symbol don't match, because using old hugo symbols: 263
Could fix this by mapping to approved symbols, but not important for this purpose since all those are in dump


In [76]:
unimerge.columns

Index(['enst_id', 'gene_name', 'refseq_id', 'ccds_id', 'hgnc_symbol',
       'uniprot_not_in_dump', 'uniprot_canonical_transcript',
       'ensembl_canonical_transcript'],
      dtype='object')

In [77]:
criteria = (~unimerge.hgnc_symbol.isnull()) & unimerge.uniprot_not_in_dump & (~unimerge.ensembl_canonical_transcript.isnull())
unimerge.loc[criteria, ['enst_id']] = unimerge.loc[criteria]['ensembl_canonical_transcript']

In [78]:
len(unimerge[criteria])

281

In [79]:
# assume order is still the same as source overrides file
assert((uniprot.gene_name != unimerge.gene_name).sum() == 0)

In [80]:
unimerge[uniprot.columns].to_csv("../data/isoform_overrides_uniprot.hg19.txt",index=False,sep="\t")